# Session 4: Reactive Transport
Sergi Molins, LBNL | September 9, 2025 | Knoxville, TN

* This notebook presents a series of exercises that demonstrate how to set up and run reactive transport simulations for integrated hydrology problems with ATS.
* The problems build on the hillslope transport problem presented in the previous session.
* This notebook is mostly intended to be used as a reference to follow the exercises along with the input files provided.

### Quick links to content:

* [Exercise 1: First-order reaction in the hillslope transport problem](#Exercise-1:-First-order-reaction-in-the-hillslope-transport-problem) 
* [Exercise 2: Reduction of nitrate from fertilizer application](#Exercise-2:-Reduction-of-nitrate-from-fertilizer-application) 
    * [Exercise 2a: Develop and test conservative transport using the Alquimia PK (PFLOTRAN or CrunchFlow)](#Exercise-2:Develop-and-test-conservative-transport-using-the-Alquimia-PK-(PFLOTRAN-or-CrunchFlow)) 
    * [Exercise 2b: Develop and test the geochemical problem in PFLOTRAN or CrunchFlow](#Exercise-2b:-Develop-and-test-the-geochemical-problem-in-PFLOTRAN-or-CrunchFlow) 
    * [Exercise 2c: Put everything together](#Exercise-2c:-Put-everything-together) 

### Begin by changing to the `04_reactive_transport` directory

```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/cd 04_reactive_transport
```

## Exercise 1: First-order reaction in the hillslope transport problem

### This exercise adds a simple first-order reaction to the solute tracer transport problem of the previous session. 

**What we will learn:**

* To add sources to the Transport PK to represent simple reactions
* To use evaluator functions to provide a function for the source
* To decide (conceptually) where reactions take place
* To evaluate the need for full multicomponent geochemistry

**Files we will use:**

* Input file: first-order_hillslope_tracer.xml
* Mesh file: hillslope_mesh.exo
* Flow restart file: last checkpoint from running 01_integrated_flow/hydrology_hillslope_spinup_open.xml

### What is added and/or modified with respect to the transport problem?

**Transport source in the subsurface**

```xml
  <ParameterList name="PKs">
    ...

    <ParameterList name="subsurface transport">
      ...

      <Parameter name="source term" type="bool" value="true"/>
      <Parameter name="source key suffix" type="string" value="tracer2_rate"/>
```

**Evaluator for source (i.e. reaction rate)**

```xml
  <ParameterList name="state">
    ...

    <ParameterList name="evaluators" type="ParameterList">
      ...

      <ParameterList name="tracer2_rate" type="ParameterList">
        <Parameter name="evaluator type" type="string" value="multiplicative evaluator"/>
        <Parameter name="coefficient" type="double" value="-1.0e-3"/>
        <Parameter name="dependencies" type="Array(string)" value="{mole_fraction}"/>
      </ParameterList>
```

**Some notes**

* We have also removed ``Tracer1`` from the input with respect to the transport_tracers.xml input file (see file transport_tracer.xml) because the code does not currently support setting up [Generic Evaluators](https://amanzi.github.io/ats/stable/input_spec/state/generic.html) for multicomponent fields. (Please request the feature if you need it with a use case to go along with it!)
* Before running remember to update the location of the spinup file you generated in the input file and use the same number of cores as you did for the spinup.
* This should take up to 10 minutes to run using 1 or 2 cores in most laptops.


### Let's run the simulation:

```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ mkdir run_first_order

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ cd run_first_order

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_first_order$ nohup mpirun -n 2 ats --xml_file=../first-order_tracer.xml > log &

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_first_order$ tail -f log | grep -e Cycle
```

### Let's compare with the conservative simulations

**VisIt session**

### Questions:

* What is the impact of the reactions in the observed surface concentrations in the retention pond or at the toe slope?
* How would you add this reaction in the surface water? Is it possible?
* What if you have different units for the rate constant? 


## Exercise 2: Reduction of nitrate from fertilizer application

This exercise adds a multicomponent geochemical problem to the transport problem of the previous session. The objective is to show the use of different water compositions (geochemical conditions), one for the irrigation water, one for the rainwater and one for the initial water in the subsurface. The irrigation water contains fertilizer with elevated nitrate concentrations that will be reduced in the subsurface where a source of organic carbon is available for oxidation. 

We are building the full problem step by step to ensure that each component of the problem functions separately before assembling the full simulation (this save a lot of grief later on):

1. Exercise 2a: Ensure that we can assign the different geochemical conditions to water sources and initial/boundary conditions to the flow and transport problem when we add the Chemistry PKs to the problem.
2. Exercise 2b: Ensure that the geochemical conditions can be solved by the geochemical engines and represent the waters in our conceptual model.
3. Exercise 2c: Assemble the overall problem and resolve potential issues that emerge at this stage. 

### Exercise 2a: Develop and test conservative transport using the Alquimia PK (PFLOTRAN or CrunchFlow)

When setting up an Integrated Reaction PK, it is strongly recommended that a conservative transport simulation is performed before where no reactions are included but where the geochemical input files are used. This is accomplished by not including reactions in the engine's input file. This has two main benefits:

* It makes it easy to check that geochemical conditions are passed correctly (as intended) between the engine and ATS.
* It provides an understanding of the fate of tracers added to different sources.

 This exercise replaces the solute tracer used in the transport problem in the previous session with two conservative tracers, tracer 1 and 2, that are assigned to the irrigation water and the rain water, respectively. It also sets up the appropriate initial and boundary conditions as needed.

**What we will learn:**

* To specify an integrated flow and reactive transport PK tree.
* To specify a geochemical engine and input file in the Chemistry PKs (of alquimia type).
* To specify initial conditions that use geochemical conditions from an engine.
* To specify different water sources for flow where we can apply different transport sources.
* To specify source terms for transport that use geochemical conditions from an engine.

**Files we will use:**

* Input files: transport_hillslope_pflotran.xml or transport_hillslope_crunch.xml
* PFLOTRAN input file: pflotran_dir/tracer-pflotran.in
* CrunchFlow input file: crunch_dir/tracer-crunch.in
* Mesh file: 
* Flow restart file:

### What is added and/or modified with respect to the transport problem?

**Integrated flow and reactive transport PK tree**

```xml
  <ParameterList name="cycle driver">
    ...
      
    <ParameterList name="PK tree">
      <ParameterList name="flow and transport">
        <Parameter name="PK type" type="string" value="coupled flow and transport"/>
        <ParameterList name="flow_coupler">
          <Parameter name="PK type" type="string" value="coupled water"/>
          <ParameterList name="subsurface flow">
            <Parameter name="PK type" type="string" value="richards flow"/>
          </ParameterList>
          <ParameterList name="surface flow">
            <Parameter name="PK type" type="string" value="overland flow, pressure basis"/>
          </ParameterList>
        </ParameterList>

        <ParameterList name="reactive transport" type="ParameterList">
          <Parameter name="PK type" type="string" value="surface subsurface reactive transport" />
          <ParameterList name="chemistry coupler" type="ParameterList">
            <Parameter name="PK type" type="string" value="weak MPC" />
              
            <ParameterList name="chemistry surface" type="ParameterList">
              <Parameter name="PK type" type="string" value="chemistry alquimia" />
            </ParameterList>
            <ParameterList name="chemistry subsurface" type="ParameterList">
              <Parameter name="PK type" type="string" value="chemistry alquimia" />
            </ParameterList>
          </ParameterList>

          <ParameterList name="transport_coupler">
            <Parameter name="PK type" type="string" value="surface subsurface transport"/>
            <ParameterList name="subsurface transport">
              <Parameter name="PK type" type="string" value="transport ATS"/>
            </ParameterList>
            <ParameterList name="surface transport">
              <Parameter name="PK type" type="string" value="transport ATS"/>
            </ParameterList>
          </ParameterList>

        </ParameterList>
      </ParameterList>
    </ParameterList>
```

**Integrated Flow and Reactive Transport PK**
```xml
  <ParameterList name="PKs" type="ParameterList">
    ...
      
    <ParameterList name="flow and transport">
      <Parameter name="PK type" type="string" value="coupled flow and transport"/>
      <Parameter name="PKs order" type="Array(string)" value="{flow_coupler, reactive transport}"/>
      <Parameter name="subcycle" type="Array(int)" value="{0, 1}"/>
    </ParameterList>
```

**Integrated Reactive Transport PK**
```xml
  <ParameterList name="PKs" type="ParameterList">
    ...
      
    <ParameterList name="reactive transport" type="ParameterList">
      <Parameter name="PK type" type="string" value="surface subsurface reactive transport"/>
      <Parameter name="PK name" type="string" value="reactive transport"/>
      <Parameter name="PKs order" type="Array(string)" value="{chemistry coupler, transport_coupler}"/>
      <Parameter name="transport index" type="int" value="1"/>
    </ParameterList>
```

**Integrated Chemistry PK**
```xml
  <ParameterList name="PKs" type="ParameterList">
    ...
      
    <ParameterList name="chemistry coupler" type="ParameterList">
      <Parameter name="PK type" type="string" value="weak MPC"/>
      <Parameter name="PK name" type="string" value="coupled chemistry"/>
      <Parameter name="PKs order" type="Array(string)" value="{chemistry subsurface, chemistry surface}"/>
    </ParameterList>
```

**Chemistry PK (surface)**

```xml
  <ParameterList name="PKs" type="ParameterList">
    ...

    <ParameterList name="chemistry surface" type="ParameterList">
      <Parameter name="PK type" type="string" value="chemistry alquimia"/>
      <Parameter name="engine" type="string" value="CrunchFlow"/>
      <Parameter name="engine input file" type="string" value="../crunch_dir/tracer-crunch.in"/>
      <Parameter name="domain name" type="string" value="surface"/>
      <Parameter name="saturation liquid key" type="string" value="surface-ponded_depth"/>
      <Parameter name="auxiliary data" type="Array(string)" value="{pH}"/>
      <Parameter name="debug cells" type="Array(int)" value="{0}"/>
      <ParameterList name="initial conditions" type="ParameterList">
        <ParameterList name="geochemical conditions" type="ParameterList">
          <ParameterList name="initial" type="ParameterList">
            <Parameter name="regions" type="Array(string)" value="{surface domain}"/>
          </ParameterList>
        </ParameterList>
      </ParameterList>
```

**Some notes:**

* Do the same for subsurface chemistry paying attention to the "domain name", "regions" and "saturation liquid key" parameters.
* The name of the PLOTRAN engine is "PFloTran". This may be updated in future releases. 

### Let's dive into the geochemical conditions

**Chemistry initial conditions (subsurface)**

```xml
    <ParameterList name="chemistry subsurface" type="ParameterList">
      ...

      <ParameterList name="initial conditions" type="ParameterList">
        <ParameterList name="geochemical conditions" type="ParameterList">
          <ParameterList name="initial" type="ParameterList">
            <Parameter name="regions" type="Array(string)" value="{computational domain}"/>
          </ParameterList>
        </ParameterList>
      </ParameterList>
```

**Geochemical condition in engine input file (PFLOTRAN)**

**Geochemical condition in engine input file (CrunchFlow)**

**Transport sources (surface) also use geochemical conditions**

```xml
  <ParameterList name="PKs">
    ...
      
    <ParameterList name="surface transport">
    ...

      <ParameterList name="source terms">
        <ParameterList name="geochemical" type="ParameterList">
            
          <ParameterList name="Rain condition" type="ParameterList">
            <Parameter name="water source key" type="string" value="surface-rain_water_source"/>
            <Parameter name="regions" type="Array(string)" value="{surface domain}"/>
            <Parameter name="solutes" type="Array(string)" value="{Tracer1,Tracer2}"/>
            <Parameter name="times" type="Array(double)" value="{0.0, 8640000.0}"/>
            <Parameter name="geochemical conditions" type="Array(string)" value="{rainfall,rainfall}"/>
            <Parameter name="time functions" type="Array(string)" value="{constant}"/>              
            <Parameter name="ats units [moles/m^3]" type="bool" value="true"/>
          </ParameterList>
            
          <ParameterList name="Irrigation condition" type="ParameterList">
            <Parameter name="water source key" type="string" value="surface-injection_water_source"/>
            <Parameter name="regions" type="Array(string)" value="{injection}"/>
            <Parameter name="solutes" type="Array(string)" value="{Tracer1,Tracer2}"/>
            <Parameter name="times" type="Array(double)" value="{0.0, 8640000.0}"/>
            <Parameter name="geochemical conditions" type="Array(string)" value="{fertilizer,fertilizer}"/>
            <Parameter name="time functions" type="Array(string)" value="{constant}"/>
            <Parameter name="ats units [moles/m^3]" type="bool" value="true"/>
          </ParameterList>

        </ParameterList>
      </ParameterList>
```

**Some Notes:**

**Evaluator for water sources tied to geochemical conditions**

```xml
  <ParameterList name="state">
    ...

    <ParameterList name="evaluators" type="ParameterList">
    ...
        
      <ParameterList name="surface-water_source" type="ParameterList">
        <Parameter name="evaluator type" type="string" value="additive evaluator"/>
        <Parameter name="dependencies" type="Array(string)" value="{surface-rain_water_source,surface-injection_water_source}"/>
      </ParameterList>
 
      <ParameterList name="surface-rain_water_source">
        <Parameter name="evaluator type" type="string" value="independent variable"/>
        <ParameterList name="function">
          <ParameterList name="rain source">
            <Parameter name="region" type="string" value="surface domain"/>
            <Parameter name="component" type="string" value="cell"/>
            <ParameterList name="function">
              <ParameterList name="function-tabular">
                <Parameter name="x values" type="Array(double)" value="{0, 864000}"/>
                <Parameter name="y values" type="Array(double)" value="{5.5e-7, 0.0}"/>
                <Parameter name="forms" type="Array(string)" value="{constant}"/>
              </ParameterList>
            </ParameterList>
          </ParameterList>
        </ParameterList>
      </ParameterList>

      <ParameterList name="surface-injection_water_source">
        <Parameter name="evaluator type" type="string" value="independent variable"/>
        <ParameterList name="function">
          <ParameterList name="irrigation source">
            <Parameter name="region" type="string" value="injection"/>
            <Parameter name="component" type="string" value="cell"/>
            <ParameterList name="function">
              <ParameterList name="function-tabular">
                <Parameter name="x values" type="Array(double)" value="{0.0, 864000.0}"/>
                <Parameter name="y values" type="Array(double)" value="{8.e-6, 0.0}"/>
                <Parameter name="forms" type="Array(string)" value="{constant}"/>               
              </ParameterList>
            </ParameterList>
          </ParameterList>
        </ParameterList>
      </ParameterList>
```

### Let's run the simulation:

```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ mkdir run_transport_hillslope
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ cd run_transport_hillslope

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_transport_hillslope$ nohup mpirun -n 2 ats --xml_file=../transport_hillslope_pflotran.xml > log &
                                                              or
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_transport_hillslope$ nohup mpirun -n 2 ats --xml_file=../transport_hillslope_crunch.xml > log &

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_transport_hillslope$ tail -f log | grep -e Cycle
```

### Let's look at the results

**VisIt**

### Exercise 2b: Develop and test the geochemical problem in PFLOTRAN or CrunchFlow

The geochemical calculations are performed by the geochemical solvers provided by PFLOTRAN and CrunchFlow. The user can choose whichever they are more familiar with. It is highly recommended that before assembling a full simulations with ATS, test runs are peformed with the geochemical engines as standalone reactive transport codes. This has two main benefits:

* It serves as a sandbox to develop the geochemical model more efficiently than running the coupled code 
* It ensures that the geochemical problem can be solved by the engine, isolating any potential issues with the engine before tackling the coupled problem

This exercise sets up a geochemical problem with PFLOTRAN and CrunchFlow that includes 4 geochemical conditions:
* rainwater (termed rainfall)
* irrigattion water (termed fertilizer) 
* subsurface water in organic carbon poor (shallow) regions (termed oc_poor)
* subsurface water in organic carbon rich (deep) regions, which includes solid organic matter, or SOM (termed oc_rich)

What we will learn:
* To explore the engine input files, make changes to constraints that define geochemical conditions
* To explore the engine output files to understand the composition of the solutions

Files we will use:
* PFLOTRAN input files: pflotran_dir/nitrate-surface.in, pflotran_dir/nitrate-subsurface.in 
* PFLOTRAN databaset file : pflotran_dir/db/hanford.dat
* CrunchFlow input files: crunch_dir/nitrate-surface.in, crunch_dir/nitrate-subsurface.in
* CrunchFlow database file: crunch_dir/db/database.dat

Change to your directory of choice, `pflotran_dir` or `crunch_dir`:

```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ cd pflotran_dir
```
or
```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ cd crunch_dir
```

### Let's explore the engine input files

### Let's run the geochemical input file with the respective standalone simulators: 

```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/pflotran_dir$pflotran --pflotranin nitrate-surface.in 

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/pflotran_dir$pflotran --pflotranin nitrate-subsurface.in
```
or
```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/crunch_dir$CrunchTope
``` 
           and then type nitrate-surface.in or nitrate-subsurface.in followed by RETURN



### Let's explore the engine output files

### Exercise 2c: Put everything together

Now that we have both components tested separately, we can assemble the coupled simulation by replacing the engine input files in the approaproate source and initial conditions sections.

What we will learn:

* To
* To 

Files we will use:
* Input files: reactive_hillslope_pflotran.xml or reactive_hillslope_crunch.xml
* PFLOTRAN input files: pflotran_dir/nitrate-surface.in, pflotran_dir/nitrate-subsurface.in
* PFLOTRAN databaset file : pflotran_dir/db/hanford.dat
* CrunchFlow input files: crunch_dir/nitrate-surface.in, crunch_dir/nitrate-subsurface.in
* CrunchFlow database file: crunch_dir/db/database.dat
* Mesh file:
* Flow restart:

### What is added and/or modified with respect to the transport problem from Exercise 2a?

**Subsurface chemistry (PFLOTRAN)**

* update input file with the one we just checked
* add mineral to the PK
* specify different initial conditions to shallow (organic carbon poor) vs deep (organic carbon rich) zones

```xml
  <ParameterList name="PKs" type="ParameterList">
    ...
      
    <ParameterList name="chemistry subsurface" type="ParameterList">
      <Parameter name="PK type" type="string" value="chemistry alquimia"/>
      <Parameter name="engine" type="string" value="PFloTran"/>
      <Parameter name="engine input file" type="string" value="../pflotran_dir/nitrate-subsurface.in"/>
      <Parameter name="domain name" type="string" value="domain"/>
      <Parameter name="minerals" type="Array(string)" value="{SOM}"/>        
      <Parameter name="auxiliary data" type="Array(string)" value="{pH}"/>
      <Parameter name="debug cells" type="Array(int)" value="{99}"/> 

      <ParameterList name="initial conditions" type="ParameterList">
        <ParameterList name="geochemical conditions" type="ParameterList">
          <ParameterList name="oc_poor" type="ParameterList">
            <Parameter name="regions" type="Array(string)" value="{shallow}"/>
          </ParameterList>
          <ParameterList name="oc_rich" type="ParameterList">
            <Parameter name="regions" type="Array(string)" value="{deep}"/>
          </ParameterList>            
        </ParameterList>
      </ParameterList>
```

**Mineral initial conditions**

Note: these are required for now but dummy values are provided. Actual values come from the conditions. This may change in the future.

```xml
  <ParameterList name="state" type="ParameterList">
    ...

    <ParameterList name="initial conditions" type="ParameterList">
      ...

      <ParameterList name="mineral_volume_fractions" type="ParameterList">
        <ParameterList name="function" type="ParameterList">
          <ParameterList name="All" type="ParameterList">
            <Parameter name="region" type="string" value="computational domain"/>
            <Parameter name="component" type="string" value="cell"/>
            <ParameterList name="function" type="ParameterList">
              <Parameter name="number of dofs" type="int" value="1"/>
              <Parameter name="function type" type="string" value="composite function"/>
              <ParameterList name="dof 1 function" type="ParameterList">
                <ParameterList name="function-constant" type="ParameterList">
                  <Parameter name="value" type="double" value="0.00001"/> <!--dummy-->
                </ParameterList>
              </ParameterList>
            </ParameterList>
          </ParameterList>
        </ParameterList>
      </ParameterList>

      <ParameterList name="mineral_specific_surface_area" type="ParameterList">
        <ParameterList name="function" type="ParameterList">
          <ParameterList name="All" type="ParameterList">
            <Parameter name="region" type="string" value="computational domain"/>
            <Parameter name="component" type="string" value="cell"/>
            <ParameterList name="function">
              <Parameter name="number of dofs" type="int" value="1"/>
              <Parameter name="function type" type="string" value="composite function"/>
              <ParameterList name="dof 1 function">
                <ParameterList name="function-constant">
                  <Parameter name="value" type="double" value="0.0"/> <!--dummy-->
                </ParameterList>
              </ParameterList>
            </ParameterList>
          </ParameterList>
        </ParameterList>
      </ParameterList>

      <ParameterList name="mineral_rate_constant">
        <ParameterList name="function">
          <ParameterList name="All">
            <Parameter name="region" type="string" value="computational domain"/>
            <Parameter name="component" type="string" value="cell"/>
            <ParameterList name="function">
              <Parameter name="number of dofs" type="int" value="1"/>
              <Parameter name="function type" type="string" value="composite function"/>
              <ParameterList name="dof 1 function">
                <ParameterList name="function-constant">
                  <Parameter name="value" type="double" value="0.0"/> <!--dummy-->
                </ParameterList>
              </ParameterList>
            </ParameterList>
          </ParameterList>
        </ParameterList>
      </ParameterList>
```

**Update the name of the transported components and sources in transport PKs**

Note: make sure that the component names are given in the same order as in the engine input file. We are currently not checking. 
    
```xml    
  <ParameterList name="transport_coupler">
    ...
    
    <ParameterList name="transport_coupler">
      <Parameter name="PK type" type="string" value="surface subsurface transport"/>
      <Parameter name="PKs order" type="Array(string)" value="{subsurface transport, surface transport}"/>
      <Parameter name="component names" type="Array(string)" value="{DOM1,O2(aq),NO3-,HCO3-,H+,N2(aq)}"/>
    </ParameterList>

    <ParameterList name="subsurface transport">
      <Parameter name="PK type" type="string" value="transport ATS"/>
      <Parameter name="component names" type="Array(string)" value="{DOM1,O2(aq),NO3-,HCO3-,H+,N2(aq)}"/>
    ...

    <ParameterList name="surface transport">
      <Parameter name="PK type" type="string" value="transport ATS"/>
      <Parameter name="domain name" type="string" value="surface"/>
      <Parameter name="water source in meters" type="bool" value="true"/>
      <Parameter name="component names" type="Array(string)" value="{DOM1,O2(aq),NO3-,HCO3-,H+,N2(aq)}"/>
    ...

      <ParameterList name="source terms">
        <ParameterList name="geochemical" type="ParameterList">
          <ParameterList name="rain water condition" type="ParameterList">
            <Parameter name="water source key" type="string" value="surface-rain_water_source"/>
            <Parameter name="regions" type="Array(string)" value="{surface domain}"/>
            <Parameter name="solutes" type="Array(string)" value="{DOM1,O2(aq),NO3-,HCO3-,H+,N2(aq)}"/>
          ...
              
          <ParameterList name="irrigation condition" type="ParameterList">
            <Parameter name="water source key" type="string" value="surface-injection_water_source"/>
            <Parameter name="regions" type="Array(string)" value="{injection}"/>
            <Parameter name="solutes" type="Array(string)" value="{DOM1,O2(aq),NO3-,HCO3-,H+,N2(aq)}"/>
          ...
```

### Let's run the simulation:

```sh
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ mkdir run_reactive_hillslope
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport$ cd run_reactive_hillslope

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_reactive_hillslope$ nohup mpirun -n 2 ats --xml_file=../reactive_hillslope_pflotran.xml > log &
                                                              or
(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_reactive_hillslope$ nohup mpirun -n 2 ats --xml_file=../reactive_hillslope_crunch.xml > log &

(base) ats_sc_user@e2614fb043da:~/work/ats-short-course/04_reactive_transport/run_reactive_hillslope$ tail -f log | grep -e Cycle
```